In [24]:
# Implementing the same Surname classifier using CNN

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import re
import os
from torch.utils.data import DataLoader
from argparse import Namespace
import torch
import torch.nn as nn
import torch.optim as optim
# Version descrepancies on this one
from tqdm.notebook import tqdm as notebook

In [14]:
sys.path.append("../cnn_helper/")

In [15]:
%load_ext autoreload
%autoreload 2

from cnn_classifier import SurnameClassifier
from cnn_utils import ModelUtils
from cnn_dataset import SurnameDataset

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
args = Namespace(
# Data and path info
surname_csv = "../data/surnames_with_splits.csv",
vectorizer_file = "vectorizer.json",
model_state_file = "model.pth",
save_dir = "../data/model_state",
# Model Hyperparameters
hidden_dim = 100,
num_channels = 256,
# Training Hyper Parameters
seed = 1337,
batch_size = 128,
num_epochs = 100,
early_stopping_criteria=5,
learning_rate = 0.001,
dropout_p=0.1,
# Runtime operations
cuda = False,
reload_from_files=False,
expand_filepaths_to_save_dir=True,
catch_keyboard_interrupt=True)

In [17]:
# Expands filepath if none available

if args.expand_filepaths_to_save_dir:
    args.vectorizer_file = os.path.join(args.save_dir,
                                        args.vectorizer_file)

    args.model_state_file = os.path.join(args.save_dir,
                                         args.model_state_file)
    
    print("Expanded filepaths: ")
    print("\t{}".format(args.vectorizer_file))
    print("\t{}".format(args.model_state_file))
    
# Check CUDA
if not torch.cuda.is_available():
    args.cuda = False

Expanded filepaths: 
	../data/model_state\vectorizer.json
	../data/model_state\model.pth


In [18]:
# Check CUDA
if torch.cuda.is_available():
    args.cuda = True
    
args.device = torch.device("cuda" if args.cuda else "cpu")

In [19]:
ModelUtils.set_seed_everywhere(args.seed,args.cuda)
ModelUtils.handle_dirs(args.save_dir)

In [20]:
# Model Initializations
if args.reload_from_files:
    print("Reloading !")
    dataset = SurnameDataset.load_dataset_and_load_vectorizer(args.surname_csv,
                                                              args.vectorizer_file)
    
else:
    print("Creating fresh!")
    dataset = SurnameDataset.load_dataset_and_make_vectorizer(args.surname_csv)
    dataset.save_vectorizer(args.vectorizer_file)
    
vectorizer = dataset.get_vectorizer()
classifier = SurnameClassifier(initial_num_channels = len(vectorizer.surname_vocab),
                              num_classes = len(vectorizer.nationality_vocab),
                              num_channels = args.num_channels)


Creating fresh!


In [9]:
classifier = classifier.to(args.device)
dataset.class_weights = dataset.class_weights.to(args.device)

# Whyn am I feeding class weights to loss function ??
loss_func = nn.CrossEntropyLoss(weight = dataset.class_weights)
optimizer = optim.Adam(classifier.parameters(),lr = args.learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                                 mode='min',factor=0.5,patience=1)

train_state = ModelUtils.make_train_state(args)

In [25]:
epoch_bar = notebook(desc = 'training routine',
                         total = args.num_epochs,
                         position = 0
                         )
dataset.set_split('train')
train_bar = notebook(desc = 'split=train',
                         total = dataset.get_num_batches(args.batch_size),
                         position=1,
                         leave=True)

dataset.set_split('validation')
val_bar = notebook(desc = 'split=validation',
                         total = dataset.get_num_batches(args.batch_size),
                         position=1,
                         leave=True)


try:    
    for epoch_index in range(args.num_epochs):
        # for the first iteration train_state gets default values
        train_state['epoch_index'] = epoch_index
        
        # dataset mode : training
        dataset.set_split('train')
        batch_generator = ModelUtils.generate_batches(dataset,batch_size=args.batch_size,device=args.device)
        
        #running loss and acc will reset to zero
        running_loss = 0.0
        running_acc = 0.0
        classifier.train()
        
        for batch_index, batch_dict in enumerate(batch_generator):
            optimizer.zero_grad()
            
            # predicting value just by passing value to classifier. 
            y_pred = classifier(batch_dict['x_surname'])
            loss = loss_func(y_pred, batch_dict['y_nationality'])
            loss_t = loss.item()
            
            ## why running loss needs to be divided by batch index?
            running_loss += (loss_t - running_loss) / (batch_index + 1)
            
            loss.backward()
            
            optimizer.step()
            
            # compute running accuracy
            acc_t = ModelUtils.compute_accuracy(y_pred,batch_dict['y_nationality'])
            
            # why divide by batch index?
            running_acc += (acc_t - running_acc)/(batch_index + 1)
            
            # update the running loss and running accuracy for training bar. 
            train_bar.set_postfix(loss=running_loss,acc=running_acc, epoch = epoch_index)
            train_bar.update()
        train_state['train_loss'].append(running_loss)
        train_state['train_acc'].append(running_acc)
        
        # dataset mode validation
        # cleaning up before validation routine
        batch_generator = ModelUtils.generate_batches(dataset,batch_size=args.batch_size,device=args.device)
        running_loss = 0
        running_acc = 0
        classifier.eval()
        dataset.set_split("validation")
        
        
        for batch_index, batch_dict in enumerate(batch_generator): 
            # predicting value just by passing value to classifier. 
            y_pred = classifier(batch_dict['x_surname'])
            loss = loss_func(y_pred, batch_dict['y_nationality'])
            loss_t = loss.item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)
            
            acc_t = ModelUtils.compute_accuracy(y_pred,batch_dict['y_nationality'])
            running_acc += (acc_t - running_acc)/(batch_index + 1)
            
            val_bar.set_postfix(loss=running_loss,acc=running_acc,epoch = epoch_index)
            val_bar.update()
            
            
        train_state['val_loss'].append(running_loss)
        train_state['val_acc'].append(running_acc)
        
        
        train_state = ModelUtils.update_train_state(args=args,model=classifier,train_state=train_state)
        
        # has to do with learning rate adjustment
        scheduler.step(train_state['val_loss'][-1])
        
        if train_state['stop_early']:
            break
        train_bar.n = 0
        val_bar.n = 0
        epoch_bar.update()
except KeyboardInterrupt:
    print("Exiting Loop")

Exiting Loop


In [32]:
# Must use map_location parameter for deserializing to cpu.
# Load pretrained weights to classifier.
classifier.load_state_dict(torch.load(train_state['model_filename'],map_location=torch.device(args.device) ))


<All keys matched successfully>

In [33]:
classifier = classifier.to(args.device)
dataset.class_weights = dataset.class_weights.to(args.device)
loss_func = nn.CrossEntropyLoss(dataset.class_weights)

dataset.set_split('test')
batch_generator = ModelUtils.generate_batches(dataset, 
                                   batch_size=args.batch_size, 
                                   device=args.device)
running_loss = 0.
running_acc = 0.
classifier.eval()

for batch_index, batch_dict in enumerate(batch_generator):
    # this is the pretrained classifier aka the model.
    y_pred =  classifier(batch_dict['x_surname'])
    
    # compute the loss
    loss = loss_func(y_pred, batch_dict['y_nationality'])
    loss_t = loss.item()
    running_loss += (loss_t - running_loss) / (batch_index + 1)

    # compute the accuracy
    acc_t = ModelUtils.compute_accuracy(y_pred, batch_dict['y_nationality'])
    running_acc += (acc_t - running_acc) / (batch_index + 1)

train_state['test_loss'] = running_loss
train_state['test_acc'] = running_acc

In [34]:
train_state['test_loss'], train_state['test_acc']

(1.780891795953115, 54.296875)

In [38]:
# All tensors must go to device
# Where apply softmax technique is finally used
def predict_nationality(surname, classifier, vectorizer, device=args.device):
    """Predict the nationality from a new surname
    
    Args:
        surname (str): the surname to classifier
        classifier (SurnameClassifer): an instance of the classifier
        vectorizer (SurnameVectorizer): the corresponding vectorizer
    Returns:
        a dictionary with the most likely nationality and its probability
    """
    vectorized_surname = vectorizer.vectorize(surname)
    vectorized_surname = torch.tensor(vectorized_surname).unsqueeze(0)
    # must send the new tensor to device
    vectorized_surname = vectorized_surname.to(device)
    result = classifier(vectorized_surname, apply_softmax=True)

    probability_values, indices = result.max(dim=1)
    index = indices.item()

    predicted_nationality = vectorizer.nationality_vocab.lookup_index(index)
    probability_value = probability_values.item()

    return {'nationality': predicted_nationality, 'probability': probability_value}

In [ ]:
new_surname = input("Enter a surname to classify: ")
classifier = classifier.to(args.device)
prediction = predict_nationality(new_surname, classifier, vectorizer)
print("{} -> {} (p={:0.2f})".format(new_surname,
                                    prediction['nationality'],
                                    prediction['probability']))


In [ ]:
def predict_topk_nationality(name, classifier, vectorizer, k=5, device=args.device):
    vectorized_name = vectorizer.vectorize(name)
    vectorized_name = torch.tensor(vectorized_name).unsqueeze(dim=0)
    vectorized_name = vectorized_name.to(device)
    prediction_vector = classifier(vectorized_name, apply_softmax=True)
    probability_values, indices = torch.topk(prediction_vector, k=k)
    
    # returned size is 1,k
    # cpu conversion is necessary because tensor object cannot directly interact with numpy
    probability_values = probability_values.detach().cpu().numpy()[0]
    indices = indices.detach().cpu().numpy()[0]
    
    results = []
    for prob_value, index in zip(probability_values, indices):
        nationality = vectorizer.nationality_vocab.lookup_index(index)
        results.append({'nationality': nationality, 
                        'probability': prob_value})
    
    return results

In [ ]:
new_surname = input("Enter a surname to classify: ")
classifier = classifier.to(args.device)

k = int(input("How many of the top predictions to see? "))
if k > len(vectorizer.nationality_vocab):
    print("Sorry! That's more than the # of nationalities we have.. defaulting you to max size :)")
    k = len(vectorizer.nationality_vocab)
    
predictions = predict_topk_nationality(new_surname, classifier, vectorizer, k=k)

print("Top {} predictions:".format(k))
print("===================")
for prediction in predictions:
    print("{} -> {} (p={:0.2f})".format(new_surname,
                                        prediction['nationality'],
                                        prediction['probability']))